In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.impute import KNNImputer

In [7]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [8]:
df = pd.read_csv("../Data_Preprocessing/df_nnz_Vmax_clean.csv")

In [9]:
df

,X,IR,pot2,ph,temp,dstr,cryst,lgCmin,lgCmax,lgCconst,...,EState_VSA6_log,EState_VSA4_log,SMR_VSA7_log,Complexity1_log,TPSA_log,TPSA1_log,TPSA2_log,MaxEStateIndex.1_log,MaxEStateIndex.2_log,Vmaxlog
0,2.757,1.071,-0.05,4.00,37.0,3,7.0,0.176091,0.176091,2.000000,...,0.500000,2.552693,2.497497,5.420535,3.618993,3.951244,3.701302,1.790549,1.791759,0.138713
1,2.757,1.071,-0.05,4.00,37.0,3,7.0,0.176091,0.176091,2.000000,...,0.500000,2.552693,2.497497,5.420535,3.618993,3.951244,3.701302,1.790549,1.791759,-0.577903
2,2.757,1.071,-0.05,4.00,37.0,3,7.0,-0.301030,1.397940,2.000000,...,0.500000,2.552693,2.497497,0.000000,3.618993,3.701302,3.951244,1.791759,1.790549,0.922362
3,2.757,1.071,-0.05,4.00,37.0,3,7.0,-0.301030,1.397940,2.000000,...,0.500000,2.552693,2.497497,0.000000,3.618993,3.701302,3.951244,1.791759,1.790549,-0.358328
4,2.757,1.071,-0.05,4.00,37.0,3,7.0,-0.698970,2.000000,1.176091,...,0.500000,2.552693,2.497497,5.420535,3.618993,3.951244,3.701302,1.790549,1.791759,0.011655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,2.280,1.350,1.20,7.00,41.4,3,6.8,-1.221849,-0.619789,0.000000,...,0.500000,0.000000,0.000000,5.420535,4.883786,3.951244,3.823847,1.790549,0.000000,-5.821023
1055,2.280,1.350,1.20,3.00,33.0,1,6.8,-2.000000,-0.795880,0.000000,...,0.960420,3.402360,4.785979,5.420535,5.928924,3.951244,3.701302,1.790549,0.000000,-3.598599
1056,2.280,1.350,1.20,7.00,33.0,1,6.8,-1.221849,-0.619789,0.000000,...,0.960420,3.402360,4.785979,5.420535,5.928924,3.951244,3.701302,1.790549,0.000000,-3.752027
1057,1.937,0.853,0.78,4.00,42.4,1,6.8,-2.000000,-0.221849,0.000000,...,0.500000,0.000000,0.000000,5.420535,3.401197,3.951244,3.823847,1.790549,0.000000,-4.389340


In [10]:
features = df.drop('Vmaxlog', axis=1)
labels = df['Vmaxlog']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)